In [150]:
import pandas as pd
import numpy as np
import matplotlib as plt
import random
from rapidfuzz import process, fuzz
import re

## Preprocessing
Books_rating.csv and books_data.csv can be downloaded from kaggle:  
<Link Text>https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews <br><br>
Books_rating.csv has all of the rating information. books_data.csv has metadata about the books. They have the common key 'Title'<br>
In preprocessing, we join the two data frames to get the category and author for each title. We then filter for fiction books only.<br>
Next, we get rid of all parentheses from the titles. These contain publisher/edition information. We then use the rapidfuzz package to find matching titles, so they can all be renamed the same thing. For instance there are many titles for 'Slaughterhouse Five' but with slight variations. We make sure that the author also matches to increase likelihood that the two titles with different spellings or punctuation are indeed the same book.

In [151]:
random.seed(42)
np.random.seed(42)

In [152]:
ratings_df = pd.read_csv('Books_rating.csv')
books_df = pd.read_csv('books_data.csv')

In [153]:
print(ratings_df.head())
print(books_df.head())

           Id                           Title  Price         User_id  \
0  1882931173  Its Only Art If Its Well Hung!    NaN   AVCGYZL8FQQTD   
1  0826414346        Dr. Seuss: American Icon    NaN  A30TK6U7DNS82R   
2  0826414346        Dr. Seuss: American Icon    NaN  A3UH4UZ4RSVO82   
3  0826414346        Dr. Seuss: American Icon    NaN  A2MVUWT453QH61   
4  0826414346        Dr. Seuss: American Icon    NaN  A22X4XUPKF66MR   

                          profileName review/helpfulness  review/score  \
0               Jim of Oz "jim-of-oz"                7/7           4.0   
1                       Kevin Killian              10/10           5.0   
2                        John Granger              10/11           5.0   
3  Roy E. Perry "amateur philosopher"                7/7           4.0   
4     D. H. Richards "ninthwavestore"                3/3           4.0   

   review/time                                   review/summary  \
0    940636800           Nice collection of Julie Strai

In [154]:
#merge the categories and authors columns with the ratings_df
df = pd.merge(ratings_df, books_df[['Title', 'categories', 'authors']], on='Title', how='left')
print(df.head())

           Id                           Title  Price         User_id  \
0  1882931173  Its Only Art If Its Well Hung!    NaN   AVCGYZL8FQQTD   
1  0826414346        Dr. Seuss: American Icon    NaN  A30TK6U7DNS82R   
2  0826414346        Dr. Seuss: American Icon    NaN  A3UH4UZ4RSVO82   
3  0826414346        Dr. Seuss: American Icon    NaN  A2MVUWT453QH61   
4  0826414346        Dr. Seuss: American Icon    NaN  A22X4XUPKF66MR   

                          profileName review/helpfulness  review/score  \
0               Jim of Oz "jim-of-oz"                7/7           4.0   
1                       Kevin Killian              10/10           5.0   
2                        John Granger              10/11           5.0   
3  Roy E. Perry "amateur philosopher"                7/7           4.0   
4     D. H. Richards "ninthwavestore"                3/3           4.0   

   review/time                                   review/summary  \
0    940636800           Nice collection of Julie Strai

In [155]:
#make smaller dataframe with only the columns we need
df = df.loc[:,['Title','User_id','review/score','authors','categories']]
df = df[df['categories'] == "['Fiction']"]
df.count()

Title           824439
User_id         657511
review/score    824439
authors         824028
categories      824439
dtype: int64

In [156]:
#get rid of parentheses in book titles
df['Title'] = df.loc[:,'Title'].str.replace(r"\s*\(.*?\)", "", regex=True)
df.head()

,Title,User_id,review/score,authors,categories
14,Whispers of the Wicked Saints,A3Q12RK71N74LB,1.0,['Veronica Haddon'],['Fiction']
15,Whispers of the Wicked Saints,A1E9M6APK30ZAU,4.0,['Veronica Haddon'],['Fiction']
16,Whispers of the Wicked Saints,AUR0VA5H0C66C,1.0,['Veronica Haddon'],['Fiction']
17,Whispers of the Wicked Saints,A1YLDZ3VHR6QPZ,5.0,['Veronica Haddon'],['Fiction']
18,Whispers of the Wicked Saints,ACO23CG8K8T77,5.0,['Veronica Haddon'],['Fiction']


In [157]:
#remove na values
print(df.isna().sum())
df = df.dropna()
df.shape

Title                0
User_id         166928
review/score         0
authors            411
categories           0
dtype: int64


(657190, 5)

In [158]:
#replace different titles for the same books with a single title.
#!!! This code block was generated by chatGPT !!!

unique_titles = df['Title'].unique()
canonical_map = {}

for title in unique_titles:
    #checks if this title is already the offical accepted spelling/punctuation of the title
    if title in canonical_map:
        continue
    #finds other titles similar to the current title
    matches = process.extract(title, unique_titles, scorer=fuzz.token_sort_ratio, score_cutoff=0.85)
    for match, score, _ in matches:
        #if the similarity is high enough, maps the matching title to the accepted official title
        canonical_map[match] = title

# Step 3: Replace titles using the mapping
df['Title'] = df['Title'].map(canonical_map)

In [159]:
#remove duplicate ratings
df.drop_duplicates(subset=['User_id','Title'], keep='first', inplace=True)
df.shape

(491821, 5)

In [160]:
#see number of unique uesers and titles. We need make these a more manageable size
print(df['Title'].unique().size)
print(df['User_id'].unique().size)

8454
268646


### Filtering with random books then users with enough ratings

In [166]:
book_rating_counts = df['Title'].value_counts()
books_with_ratings = book_rating_counts[book_rating_counts > 50].index
filtered_df = df[df['Title'].isin(books_with_ratings)]
print(filtered_df.shape)
print(f'users: {filtered_df['User_id'].unique().size}')
print(f'books: {filtered_df['Title'].unique().size}')
print(f'matrix dimensions{filtered_df.shape}')

(393216, 5)
users: 223226
books: 2150
matrix dimensions(393216, 5)


In [167]:
user_rating_counts = filtered_df['User_id'].value_counts()
users_with_ratings = user_rating_counts[user_rating_counts > 20].index
filtered_df = filtered_df[filtered_df['User_id'].isin(users_with_ratings)]
print(filtered_df.shape)
print(f'users: {filtered_df['User_id'].unique().size}')
print(f'books: {filtered_df['Title'].unique().size}')
print(f'matrix dimensions{filtered_df.shape}')

(45686, 5)
users: 1066
books: 2135
matrix dimensions(45686, 5)


In [168]:
df_pivot = filtered_df.pivot_table(index='User_id', columns='Title', values='review/score', aggfunc = 'max')
df_pivot.head()

Title,"""Chosen"" Classics: Round the World in Eighty Days","""D"" is for Deadbeat","""Mildred Pierce","""Ra Force Rising"": Brother G","""Thirty years in hell""; or, ""From darkness to light",'Tis The Season: The Choice\First Fruits\A New Year; A New Beginning,'Tis the Season to Be Murdered,... Summer moonshine,...Arrow pointing nowhere,1 Ragged Ridge Road,...,Zane's Gettin' Buck Wild: Sex Chronicles II,Zane's Skyscraper: A Novel,Zazie dans le Mtro,Zen Attitude,Zia,Zorba the Greek,green valley,never too much,our davie pepper,the Picture of Dorian Gray
User_id,,,,,,,,,,,,,,,,,,,,,
A106016KSI0YQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A106E1N0ZQ4D9W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10T0OW97SFBB,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10Y3OZWENAQ6W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1129LM24YWSZV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Save the pivot table with all ratings to csv

In [171]:
df_pivot.to_csv('/home/ldrich/Summer2025BHT/DS_Workflow_Applications/recommender_project/BHTRecommenderProject/ratings_matrix.csv')

#### Save the basic metadata for the subset of books we are using

In [172]:
filtered_df.to_csv('/home/ldrich/Summer2025BHT/DS_Workflow_Applications/recommender_project/BHTRecommenderProject/filtered_ratings.csv')

212404
221998
